In [1]:
import pandas as pd
import glob
import numpy as np
import re

In [2]:
df1=pd.read_csv('Concat_df.csv')
df2=pd.read_csv('new_data.csv')

In [3]:
dfs = [df1, df2]

# Concaténez les DataFrames de la liste en utilisant la fonction concat()
data = pd.concat(dfs)

# Réinitialisez l'index du DataFrame concaténé
data.reset_index(drop=True, inplace=True)


In [4]:
data['Nom_Prenom']=data['Nom_Prenom'].str.upper()

In [5]:
data['Associé']=data['Associé'].fillna('Remplaçant')

In [6]:
data['Date_Heure_Fin'] = pd.to_datetime(data['Date_Heure_Fin'])
data['Date_Heure_Début'] = pd.to_datetime(data['Date_Heure_Début'])

data['Durée'] = data['Date_Heure_Fin'] - data['Date_Heure_Début']

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47521 entries, 0 to 47520
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   Source            47521 non-null  object         
 1   Nom_Prenom        47521 non-null  object         
 2   Date              47521 non-null  object         
 3   Heure_Début       47521 non-null  object         
 4   Heure_Fin         47521 non-null  object         
 5   Horaire           47521 non-null  object         
 6   Date_Heure_Fin    47521 non-null  datetime64[ns] 
 7   Date_Heure_Début  47521 non-null  datetime64[ns] 
 8   Associé           47521 non-null  object         
 9   mail              33303 non-null  object         
 10  Site              33339 non-null  object         
 11  Jour/Nuit         33880 non-null  object         
 12  Chef              6362 non-null   object         
 13  Formation         6270 non-null   object         
 14  Durée 

In [8]:
data=data.drop_duplicates()

In [9]:
data.sample(3)

,Source,Nom_Prenom,Date,Heure_Début,Heure_Fin,Horaire,Date_Heure_Fin,Date_Heure_Début,Associé,mail,Site,Jour/Nuit,Chef,Formation,Durée
1939,CDS,HERMANT JULIEN,14/01/2021,13:00:00,18:00:00,13:00:00 - 18:00:00,2021-01-14 18:00:00,2021-01-14 13:00:00,Remplaçant,NaN,NaN,NaN,NaN,NaN,0 days 05:00:00
38980,PDS,GHELIS NIL,29/04/2023,15:00:00,08:00:00,15:00:00 - 08:00:00,2023-04-30 08:00:00,2023-04-29 15:00:00,Associé,n.ghelis@imadis.fr,Lyon,Nuit,NaN,NaN,0 days 17:00:00
18526,CDS,ARNAUD FRANÇOIS,17/10/2022,09:00:00,14:00:00,09:00:00 - 14:00:00,2022-10-17 14:00:00,2022-10-17 09:00:00,Associé,f.arnaud@imadis.fr,Marseille,Journée,NaN,NaN,0 days 05:00:00


In [10]:
data.shape

(45201, 15)

In [11]:
data.isna().sum()

Source                  0
Nom_Prenom              0
Date                    0
Heure_Début             0
Heure_Fin               0
Horaire                 0
Date_Heure_Fin          0
Date_Heure_Début        0
Associé                 0
mail                13899
Site                13875
Jour/Nuit           13641
Chef                39059
Formation           38967
Durée                   0
dtype: int64

In [12]:
data['Durée']=data['Durée'].astype(str)

In [13]:
data['Durée']=data['Durée'].str.split().str[2]

In [14]:
#data['Durée'] = data['Durée'].str.replace("+ ", "")
data['Durée'] = data['Durée'].str.replace("+", "")

C:\Users\d.mandon\AppData\Local\Temp\ipykernel_13408\1769604156.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Durée'] = data['Durée'].str.replace("+", "")


In [15]:
data['Durée'] = pd.to_timedelta(data['Durée'])

# Calculez la durée en heures décimales
data['Durée'] = data['Durée'].dt.total_seconds() / 3600

In [16]:
data.sample(5)

,Source,Nom_Prenom,Date,Heure_Début,Heure_Fin,Horaire,Date_Heure_Fin,Date_Heure_Début,Associé,mail,Site,Jour/Nuit,Chef,Formation,Durée
17820,CDS,RENOSI GUILLAUME,19/07/2022,08:00:00,13:00:00,08:00:00 - 13:00:00,2022-07-19 13:00:00,2022-07-19 08:00:00,Associé,g.renosi@imadis.fr,Lyon,Journée,Chef de CDS,NaN,5.0
38024,PDS,SCHREINER CAMILLE,28/03/2023,20:00:00,08:00:00,20:00:00 - 08:00:00,2023-03-29 08:00:00,2023-03-28 20:00:00,Associé,c.schreiner@imadis.fr,Lyon,Nuit,NaN,NaN,12.0
28702,PDS,RENOSI GUILLAUME,14/05/2022,08:30:00,12:30:00,08:30:00 - 12:30:00,2022-05-14 12:30:00,2022-05-14 08:30:00,Associé,g.renosi@imadis.fr,Lyon,Journée,NaN,F2,4.0
31082,PDS,MORCET-DELATTRE THIBAUD,05/08/2022,20:00:00,08:00:00,20:00:00 - 08:00:00,2022-08-06 08:00:00,2022-08-05 20:00:00,Thèsé,t.morcet-delattre@imadis.fr,Rennes,Nuit,NaN,NaN,12.0
31117,PDS,RUIMY ALEXIS,07/08/2022,08:00:00,16:00:00,08:00:00 - 16:00:00,2022-08-07 16:00:00,2022-08-07 08:00:00,Interne,a.ruimy@imadis.fr,Marseille,Journée,NaN,NaN,8.0


In [17]:
data.to_csv('BDD.csv', index=False)